# LLM humor detection with Subspace based metric

In [1]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

MODEL_ID = "google/gemma-2-2b-it"

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
    output_hidden_states=True  # Enable hidden states output
)

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [2]:
ground_truth = pd.read_csv('../data/stand_up_dataset/standup_data.csv')
transcript = pd.read_csv('../data/stand_up_dataset/standup_transcripts.csv')

In [10]:
INSTRUCTIONS = [
    "Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    "The following is a stand-up comedy transcript. When performed in front of a live audience, which jokes do you think made the audience laugh?  List of quotes:",
    "You are a person who enjoys aggressive humor. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    "You are a person who enjoys self-enhancing humor. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "You are a person who enjoys self-deprecating humor. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "You are a person who enjoys dark humor. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "You are a person who enjoys affiliative humor. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "The following is a stand-up comedy transcript. What are the funniest punchlines from the transcript. List of quotes:",
    # "Below is a transcript from a stand-up comedy routine. Analyze the transcript and extract the quotes that are most likely to have made the audience laugh. List of quotes:",
    # "The following is a stand-up comedy transcript. When preformed in front of a live audience, which jokes do you think made the audience laugh? List of quotes:",
    # "Pretend that you are a stand-up comedian reading the following stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "Pretend that you are a stand-up comedy fan reading the following stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    # "Pretend that you are a stand-up comedy critic reading the following stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:",
    #"Analyze the stand-up comedy transcript below. Which lines and punchlines do you think delivered the biggest laughs to the audience? List of quotes:", 
    #"As a person who enjoys witty, intellectual humor, extract the key humorous lines and punchlines from this stand-up comedy transcript. Focus on the quotes that demonstrate clever wordplay or insights. List of quotes:",
    #"This is a transcript from a stand-up routine. Identify the lines and punchlines that likely had the strongest comedic impact during the performance. List of quotes:",
    #"Pretend you're an audience member at this stand-up show. Which lines do you think got the biggest laughs? Focus on key moments of humor. List of quotes:",
    #"This is a transcript of a live stand-up performance. Which quotes do you believe would have resonated the most with the audience? Focus on key punchlines. List of quotes:",
    #"Imagine you are a comedian reviewing this stand-up routine. Identify the funniest moments and lines where the punchlines landed the hardest. List of quotes:",
    #"Read through the stand-up comedy transcript and extract the lines that best capture the humor and timing of the performance. Focus on punchlines that likely had the audience laughing. List of quotes:",
    #"This is a stand-up comedy transcript. Analyze the content and extract the lines that most effectively build up to or deliver punchlines. List of quotes:",
    #"Pretend you're watching this performance live. What do you think were the standout comedic lines and punchlines that elicited the loudest laughs? List of quotes:",
    #"Imagine you are writing a review of this stand-up performance. What lines and punchlines would you highlight as the funniest moments? List of quotes:"
]

CONTENTS = [
    "",
    "Sure, here are the key humorous lines:",
    "Here are some lines and punchlines that could be funny:",
    "Got it! Here are the main punchlines and comedic highlights:",
    "Here’s a selection of the funniest quotes from the transcript:",
    "I've picked out the key humorous moments for you:",
    "Below are the standout lines and punchlines from the performance:",
    "Here's a breakdown of the top quotes that likely got the biggest laughs:",
    "Take a look at these key comedic lines from the routine:",
    "Here’s a list of the most memorable punchlines from the set:",
    "Check out these quotes—some of the best comedic moments from the transcript:",
    "Here are the funniest moments and punchlines I found in the transcript:",
    "Here’s what I’ve identified as the standout lines and punchlines in this comedy routine:" 
]

In [8]:
gt = ground_truth.groupby("comedian")["sentence"].apply(list).apply(lambda sentences: "\n".join([f"{i + 1}. {s}" for i, s in enumerate(sentences)]))
df = transcript.set_index("comedian").join(gt).rename(columns={"sentence": "ground_truth"})

df["instruction"] = [INSTRUCTIONS] * len(df)
df = df.explode("instruction")
df["content"] = [CONTENTS] * len(df)
df = df.explode("content")

def gt_chat_template(row):
    return tokenizer.apply_chat_template([
        # {"role": "system", "content": ""},
        {"role": "user", "content": row["instruction"] + "\n" + row["transcript"]},
        {"role": "assistant", "content": row["content"] + "\n" + row["ground_truth"]},
    ], tokenize=False)

df["gt_input"] = df.apply(gt_chat_template, axis=1)

def model_chat_template(row):
    return tokenizer.apply_chat_template([
        # {"role": "system", "content": ""},
        {"role": "user", "content": row["instruction"] + "\n" + row["transcript"]},
    ], tokenize=False)

df["model_input"] = df.apply(model_chat_template, axis=1)

df

(2448, 4)

In [5]:
from tqdm import tqdm

# use unembedding tokenization form
def get_gt_representation(batch_of_strs: list[str], number_of_tokens: int = 128) -> torch.Tensor:
    inputs = tokenizer(batch_of_strs, return_tensors="pt", padding=True, truncation=False).to(model.device)
    with torch.inference_mode():
        return model(**inputs).hidden_states[-1][:, -number_of_tokens:].flatten(1)

gt_representations = {
    comedian: get_gt_representation(batch.tolist())
    for comedian, batch in tqdm(df.groupby("comedian")["model_input"])
}

100%|██████████| 51/51 [00:04<00:00, 10.49it/s]


In [6]:
def get_output_representation(batch_of_strs: list[str], number_of_tokens: int = 128) -> torch.Tensor:
    inputs = tokenizer(batch_of_strs, return_tensors="pt", padding=True, truncation=False).to(model.device)
    with torch.inference_mode():
        return model(input_ids=model.generate(**inputs, max_new_tokens=128)).hidden_states[-1][:, -number_of_tokens:].flatten(1)

output_representations = {
    comedian: get_output_representation(batch.tolist())
    for comedian, batch in tqdm(df.groupby("comedian")["gt_input"])
}

100%|██████████| 51/51 [01:49<00:00,  2.14s/it]


In [7]:
gt_representations["Ali_Wong"].shape

torch.Size([4, 294912])

In [8]:
output_representations["Ali_Wong"].shape

torch.Size([4, 294912])

In [22]:
def make_subspace(data: torch.FloatTensor, q: int = 3) -> torch.Tensor:
    data = torch.nn.functional.normalize(data, p=2, dim=-1)
    data = data - data.mean(0, keepdim=True)
    *_, Vh = torch.pca_lowrank(data, q=q)
    return Vh

scores = {}
for comedian in tqdm(gt_representations.keys()):
    gt_reference_subspace = make_subspace(gt_representations[comedian].float())
    out_reference_subspace = make_subspace(output_representations[comedian].float())

    A = gt_reference_subspace.mT @ out_reference_subspace
    scores[comedian] = (A.mT @ A).trace() / A.shape[0]

100%|██████████| 51/51 [00:00<00:00, 239.06it/s]


In [ ]:
scores = pd.DataFrame(scores)

In [34]:
df.loc["Ali_Wong", "gt_input"].tolist()

["<bos><start_of_turn>user\nExtract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:\nThe last time I was at home in San Francisco, I was trying to help her get rid of shit. Don’t ever do that with your mom. It was like the worst experience of my life. It was so emotional. We were screaming and fighting and yelling and it all came to a climax when she refused to let go of a Texas Instruments TI-82… manual. The manual. She don’t even know… where the calculator is. Those of you under 25 probably don’t know what that calculator is. It was this calculator that bamboozled my generation. We were all required to buy it when we were in eight grade. It cost like $200. And everybody thought it was like this Judy Jetson’s laptop from the future. All because what? It could graph. It was like the Tesla of my time. And my mom got so emotional about the manual and she was like, “You never know when yo

In [30]:
data = gt_representations["Ali_Wong"].float()
data = torch.nn.functional.normalize(data, p=2, dim=-1)

(data @ make_subspace(data)).square().mean(-1).sqrt()

tensor([0.0282, 0.0282, 0.0282, 0.0282], device='cuda:0')

In [11]:
gt_reference_list = []
for i in all_representations:
    for comedian, rep in i.items():
        gt_references = torch.stack(rep)
        *_, gt_reference_subspace = torch.pca_lowrank(gt_references.float(), q=10)
        gt_reference_list.append({comedian: gt_reference_subspace})
    
# gt_reference_subspace.shape

NameError: name 'all_representations' is not defined

In [38]:
#sentences = [f"{inst}\n{text}\n" for inst, text in product(INSTRUCTIONS, TRANSCRIPTS)]

#inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(model.device)
#with torch.inference_mode():
    #outputs = model(input_ids=model.generate(**inputs, max_new_tokens=128))


# representations = torch.cat(outputs.hidden_states)[-1, -16:].flatten()
# all_representations.append(representation)